In [1]:
import gzip
from collections import defaultdict
import math
import numpy
import string
import random
import string

In [13]:
import pandas as pd
df = pd.read_csv("COGS188_project/databases/wcc(training)/chess_wc_history_moves.csv")

In [14]:
df.to_csv(
"COGS188_project/databases/wcc(training)/chess_wc_history_moves.gzip", index=False, compression="gzip"
)